<a href="https://colab.research.google.com/github/AmirMoghadamFalahi/sample_task/blob/main/model/using_haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/'My Drive'/

/content/drive/My Drive


In [3]:
import pandas as pd
import json

In [4]:
dataset = pd.read_csv('search_result_sample.csv')
dataset.head()

,id,article_datetime,article_timestamp,category,access_control,headline,link,got_single,paragraph_dic
0,1,2020-10-10,1602333132,Marketing,Subscription Required,Carmaker Honda tilts scale to trucks,/marketing/carmaker-honda-tilts-scale-trucks,True,"{""1"": ""LOS ANGELES — A year ago, the mantra be..."
1,4,2020-10-09,1602243712,Suppliers,Subscription Required,Yanfeng system attacks COVID-19 inside cars wi...,/suppliers/yanfeng-system-attacks-covid-19-ins...,True,"{""1"": ""Interior supplier Yanfeng has revealed ..."
2,6,2020-10-10,1602352090,Dealers,Subscription Required,Phone calls take new priority in pandemic,/dealers/phone-calls-take-new-priority-pandemic,True,"{""1"": ""More customers are buying cars using a ..."
3,7,2020-10-10,1602302401,Suppliers,Subscription Required,The new kink in automotive hiring: Amazon,/suppliers/new-kink-automotive-hiring-amazon,True,"{""1"": ""As if it hasn't been hard enough recrui..."
4,8,2020-10-10,1602284825,Cars & Concepts,Subscription Required,"Nissan dumps vans in U.S. and Canada, eyes new...",/cars-concepts/nissan-dumps-vans-us-and-canada...,True,"{""1"": ""Nissan is shifting strategy on commerci..."


In [5]:
dataset.columns

Index(['id', 'article_datetime', 'article_timestamp', 'category',
       'access_control', 'headline', 'link', 'got_single', 'paragraph_dic'],
      dtype='object')

In [61]:
final_dicts = []

for i, dic in enumerate(dataset.paragraph_dic):
  dic = json.loads(dic, encoding='utf-8')
  dic_lst = dic.values()
  txt = ('\n'.join(dic_lst))
  dic = {'text': txt, 'meta': {'category': dataset.category[i], 'headline': dataset.headline[i], 
                               'datetime': dataset.article_datetime[i], 'id': dataset.id[i]}}
  final_dicts.append(dic)

In [7]:
!pip install git+https://github.com/deepset-ai/haystack.git

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-g7vkrevi
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-g7vkrevi
     |████████████████████████████████| 194kB 2.6MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 225kB 7.9MB/s 
     |████████████████████████████████| 317kB 8.2MB/s 
     |████████████████████████████████| 92kB 7.4MB/s 
     |████████████████████████████████| 983kB 11.6MB/s 
     |████████████████████████████████| 5.5MB 16.4MB/s 
     |████████████████████████████████| 143kB 30.6MB/s 
     |████████████████████████████████| 225kB 39.6MB/s 
     |████████████████████████████████| 7.2MB 17.4MB/s 
     |████████████████████████████████| 3.9MB 42.1MB/s 
     |████████████████████████████████| 47.7MB 141kB/s 
     |████████████████████████████████| 1

In [39]:
cd ../..

/content


In [40]:
ls

drive/  sample_data/


In [68]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [69]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

10/13/2020 19:43:56 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.029s]
10/13/2020 19:43:56 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.013s]
10/13/2020 19:43:56 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.025s]
10/13/2020 19:43:56 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.006s]


In [70]:
document_store.write_documents(final_dicts)

10/13/2020 19:43:57 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.600s]
10/13/2020 19:43:58 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.930s]
10/13/2020 19:43:59 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.929s]
10/13/2020 19:44:00 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.986s]
10/13/2020 19:44:01 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.016s]
10/13/2020 19:44:02 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.007s]
10/13/2020 19:44:03 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.957s]
10/13/2020 19:44:04 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.016s]


In [72]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [73]:
from haystack.reader.farm import FARMReader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

10/13/2020 19:44:32 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
10/13/2020 19:44:32 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
10/13/2020 19:44:40 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
10/13/2020 19:44:49 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
10/13/2020 19:44:55 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
10/13/2020 19:44:55 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
10/13/2020 19:44:55 - INFO - farm.infer -    0 
10/13/2020 19:4

In [74]:
from haystack import Finder
finder = Finder(reader, retriever)

In [75]:
prediction = finder.get_answers(question="what companies have partnership with each other?", top_k_retriever=100, top_k_reader=50)

10/13/2020 19:44:55 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.138s]
10/13/2020 19:44:55 - INFO - haystack.retriever.sparse -   Got 100 candidates from retriever
10/13/2020 19:44:55 - INFO - haystack.finder -   Reader is looking for detailed answer in 407148 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  7.60 Batches/s]


In [76]:
prediction

{'answers': [{'answer': 'Honda Motor Co., electric vehicle startup Nikola Corp. and battery supplier LG Chem',
   'context': " Give and take.\nPartnerships with Honda Motor Co., electric vehicle startup Nikola Corp. and battery supplier LG Chem show GM's willingness to share t",
   'document_id': '5b28862b-1c0b-471f-b896-9d59ef31de03',
   'meta': {'category': 'Automakers & Suppliers',
    'datetime': '2020-09-14',
    'headline': "GM isn't traveling alone into the EV era",
    'id': 540},
   'offset_end': 117,
   'offset_end_in_doc': 159,
   'offset_start': 34,
   'offset_start_in_doc': 76,
   'probability': 0.8135743782016633,
   'score': 11.787240028381348},
  {'answer': 'Honda Motor Co., electric vehicle startup Nikola Corp. and battery supplier LG Chem',
   'context': " Give and take.\nPartnerships with Honda Motor Co., electric vehicle startup Nikola Corp. and battery supplier LG Chem show GM's willingness to share t",
   'document_id': 'a86e89b6-f5a4-48a9-96ed-1290a7adfe37',
   '

In [34]:
from haystack import Finder
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [78]:
from haystack.reader.transformers import TransformersReader

reader = TransformersReader(model="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

10/13/2020 19:48:50 - INFO - filelock -   Lock 140109809736392 acquired on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock


10/13/2020 19:48:51 - INFO - filelock -   Lock 140109809736392 released on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock


10/13/2020 19:48:52 - INFO - filelock -   Lock 140109802255752 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


10/13/2020 19:48:53 - INFO - filelock -   Lock 140109802255752 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


10/13/2020 19:48:54 - INFO - filelock -   Lock 140109942150424 acquired on /root/.cache/torch/transformers/c3f38b0676fe95e83b0eb2038453c6a79f727c44cbf566992fd9557e459abac5.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock


10/13/2020 19:48:55 - INFO - filelock -   Lock 140109942150424 released on /root/.cache/torch/transformers/c3f38b0676fe95e83b0eb2038453c6a79f727c44cbf566992fd9557e459abac5.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock


10/13/2020 19:48:56 - INFO - filelock -   Lock 140109809736392 acquired on /root/.cache/torch/transformers/e88f38f2c8bc669ef7873de68f36bf764d4f64b9833ca8401efe271aab476745.0f15800a5b4c30725c555e054e3d0262e9916635f0de9d397c30acd86c21dc73.lock


10/13/2020 19:48:56 - INFO - filelock -   Lock 140109809736392 released on /root/.cache/torch/transformers/e88f38f2c8bc669ef7873de68f36bf764d4f64b9833ca8401efe271aab476745.0f15800a5b4c30725c555e054e3d0262e9916635f0de9d397c30acd86c21dc73.lock


10/13/2020 19:48:58 - INFO - filelock -   Lock 140109796739168 acquired on /root/.cache/torch/transformers/dfa987aac92dc15d249af90a287974fd64aedb6548e287a4c031a16b06eb173c.f4565e3948d4331d7e0460adbcbdcac536e9886f24a2fad1190d6b53c231a3a3.lock


10/13/2020 19:49:25 - INFO - filelock -   Lock 140109796739168 released on /root/.cache/torch/transformers/dfa987aac92dc15d249af90a287974fd64aedb6548e287a4c031a16b06eb173c.f4565e3948d4331d7e0460adbcbdcac536e9886f24a2fad1190d6b53c231a3a3.lock


In [79]:
finder = Finder(reader, retriever)

In [80]:
prediction = finder.get_answers(question="what companies are partner?", top_k_retriever=100, top_k_reader=50)
prediction

10/13/2020 19:49:33 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.072s]
10/13/2020 19:49:33 - INFO - haystack.retriever.sparse -   Got 100 candidates from retriever
10/13/2020 19:49:33 - INFO - haystack.finder -   Reader is looking for detailed answer in 437055 chars ...
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` inste

{'answers': [{'answer': 'Renault,',
   'context': 'c vehicle as a mass-market family car. Together with alliance partner Renault, Nissan thought it could sell 1.5 million EVs by 2016 and promised new',
   'document_id': '92bc4380-3b02-4d1f-af87-9bd7900e0e43',
   'meta': {'category': 'Mobility Report Newsletter',
    'datetime': '2020-09-24',
    'headline': 'U.S. DOT offers $5 million in prizes to boost accessibility in autonomous vehicles',
    'id': 338},
   'offset_end': 2790,
   'offset_start': 2782,
   'probability': 0.9948310852050781,
   'score': None},
  {'answer': 'Renault,',
   'context': 'c vehicle as a mass-market family car. Together with alliance partner Renault, Nissan thought it could sell 1.5 million EVs by 2016 and promised new',
   'document_id': 'e76bbeab-22bd-4930-8666-36a09d351ec3',
   'meta': {'category': 'Mobility Report Newsletter',
    'datetime': '2020-09-24',
    'headline': 'U.S. DOT offers $5 million in prizes to boost accessibility in autonomous vehicles'